Notebook to generate masks from the jsons

In [ ]:
import json
import tifffile
import os
import numpy as np
import pandas as pd
import codecs
import cv2
import matplotlib.pyplot as plt
import shutil


In [ ]:
data_basepath = '/N/slate/yashjain/kaggle_data_package/kaggle_data_multiftu/data/' 
annotation_path = data_basepath + 'annotations/'
images_path = data_basepath + 'images/'

In [ ]:
save_path =  data_basepath + 'gftu_dataset_separate'

In [ ]:
import skimage.io as io
def get_mask_from_json(jsn:json,shape=(32768 , 626880)):
    
    #code from colon deepflash5
    
    data = json.load(codecs.open(jsn, 'r', 'utf-8-sig'))
#json.load(open(jsn).read().decode('utf-8-sig'))
    polys = []
    for index in range(data.__len__()):
        
        
        
        geom = [np.array([x]) for x in data[index]['geometry']['coordinates']]
        
        for g in geom:
            polys.append(g.astype(int))
        
        
            
        
    mask = np.zeros(shape)
    for i in range(len(polys)):
        if cv2.contourArea(polys[i].astype(int)) >= 10:
            cv2.fillPoly(mask, polys[i], 1)
        
    return mask

    

Sample test

In [ ]:
json_example = annotation_path + '/kidney/10606_32734_A_9_5_kidney.json'
image_example = images_path + '/kidney/10606_32734_A_9_5_kidney.tif'

In [ ]:
img = tifffile.imread(image_example)
mask = get_mask_from_json(json_example,(img.shape[0],img.shape[1]))

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(mask)

Logic working fine. Write for the whole dataset

In [ ]:
for organ in os.listdir(annotation_path):
    print(f"Extracting masks for organ = {organ}")
    if not os.path.isdir(save_path+"/"+organ):
        os.makedirs(save_path+"/"+organ+"/"+"image")
        os.makedirs(save_path+"/"+organ+"/"+"mask")
    for annotation_json in os.listdir(annotation_path+"/"+organ+"/"):
        if ".ipynb" in annotation_json:
            continue
        try:
            shutil.copyfile(images_path+organ+"/"+annotation_json.replace("json","tif"),save_path+"/"+organ+"/"+"image/"+annotation_json.replace("json","tif"))
            image = tifffile.imread(images_path+"/"+organ+"/"+annotation_json.replace("json","tif"))
            mask = get_mask_from_json(annotation_path+"/"+organ+"/"+annotation_json,(image.shape[0],image.shape[1]))
            tifffile.imwrite(save_path+"/"+organ+"/"+"mask/"+annotation_json.replace("json","tif"),np.uint8(mask*255))
        except:
            print(images_path+"/"+organ+"/"+annotation_json.replace("json","tif"))
            print("file not found")
        
        
        
    
    

RLE to png mask

In [ ]:
import pandas as pd
import tifffile
from PIL import Image

def rle2mask(rle, shape):
    '''
    mask_rle: run-length as string formatted (start length)
    shape: (height, width) of array to return 
    Returns numpy array <- 1(mask), 0(background)
    '''
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')


image_dir = "gftu_dataset/train/"
result_csv = pd.read_csv("complete_dataset_rle.csv")

#reading image name and rle from the csv
image_index_in_csv = 10

im_file_name,rle = result_csv.iloc[image_index_in_csv,0],result_csv.iloc[image_index_in_csv,1]

#deriving image shape for the rle
img_shape = tifffile.imread(image_dir+im_file_name).shape

#rle to mask conversion
mask = rle2mask(rle,(img_shape[0],img_shape[1]))

#comment this if not required
plt.imshow(mask)
plt.show()

#saving masK as png
Image.fromarray(mask*255).save("test.png")

